In [ ]:
#!/usr/bin/env python3
"""
Connect Four (7x6) with Color Effects using ANSI Codes
- No external libraries needed for the core game.
- Human = Red (X)
- Computer = Yellow (O)
"""

import random
from typing import List

from connect_four_core import (
    AI,
    COLS,
    EMPTY,
    HUMAN,
    ROWS,
    board_full,
    create_board,
    drop_piece,
    get_next_open_row,
    get_valid_locations,
    is_valid_location,
    minimax,
    winning_move,
)

# === ANSI Color Codes ===
RED = "\033[91m"
YELLOW = "\033[93m"
GREEN = "\033[92m"
CYAN = "\033[96m"
MAGENTA = "\033[95m"
GRAY = "\033[90m"
RESET = "\033[0m"


def print_board(board: List[List[int]]) -> None:
    """Display board with colored tokens."""
    print(f"\n   {'   '.join(str(c) for c in range(COLS))}")
    print("  +" + "---+" * COLS)
    for r in range(ROWS):
        row_str = ""
        for c in range(COLS):
            cell = board[r][c]
            if cell == HUMAN:
                row_str += f"| {RED}X{RESET} "
            elif cell == AI:
                row_str += f"| {YELLOW}O{RESET} "
            else:
                row_str += f"| {GRAY}·{RESET} "
        row_str += "|"
        print("  " + row_str)
        print("  +" + "---+" * COLS)
    print()


def human_turn(board: List[List[int]]) -> bool:
    while True:
        raw = input(f"{CYAN}Your move — choose a column [0–6] (or 'q' to quit): {RESET}").strip().lower()
        if raw in ("q", "quit", "exit"):
            return False
        if not raw.isdigit():
            print(f"{RED}⚠️ Please enter a number between 0 and 6.{RESET}")
            continue
        col = int(raw)
        if not is_valid_location(board, col):
            print(f"{YELLOW}⚠️ Column full or out of range. Try again.{RESET}")
            continue
        row = get_next_open_row(board, col)
        if row is None:
            print(f"{YELLOW}⚠️ Column full. Try another one.{RESET}")
            continue
        drop_piece(board, row, col, HUMAN)
        return True

def ai_turn(board: List[List[int]], depth: int = 3) -> None:
    print(f"{MAGENTA}\n🤖 Computer is thinking...{RESET}")
    score, col = minimax(board, depth, True)
    if col is None:
        valid = get_valid_locations(board)
        col = random.choice(valid) if valid else 0
    row = get_next_open_row(board, col)
    drop_piece(board, row, col, AI)
    print(f"{YELLOW}💻 Computer drops in column {col} (score: {score:.1f}).{RESET}\n")

def play_game() -> None:
    print(f"{CYAN}\n=== CONNECT FOUR ==={RESET}")
    print(f"You: {RED}X{RESET}   |   Computer: {YELLOW}O{RESET}\n")
    print(f"{GRAY}Goal: Connect four in a row horizontally, vertically, or diagonally.{RESET}\n")

    board = create_board()
    print_board(board)
    human_to_move = True

    while True:
        if human_to_move:
            moved = human_turn(board)
            if not moved:
                print(f"{GRAY}You quit. Goodbye!{RESET}")
                return
            print_board(board)
            if winning_move(board, HUMAN):
                print(f"{GREEN}🎉 You win! Incredible strategy!{RESET}")
                return
        else:
            ai_turn(board, depth=3)
            print_board(board)
            if winning_move(board, AI):
                print(f"{RED}🤖 Computer wins! Try again next time!{RESET}")
                return

        if board_full(board):
            print(f"{CYAN}🤝 It's a draw!{RESET}")
            return

        human_to_move = not human_to_move

# Note: Basic console version functions are kept for reference but not invoked.
# The GUI version (ConnectFourGUI) is the main interface and is launched in Cell 4.


## GUI Interface for Connect Four

Beautiful graphical interface using Jupyter Widgets for better gaming experience!


In [3]:
# GUI Implementation using Jupyter Widgets
import ipywidgets as widgets
from IPython.display import display, clear_output
import time

class ConnectFourGUI:
    """GUI wrapper for Connect Four game using existing game logic."""

    def __init__(self):
        self.rows = ROWS
        self.cols = COLS
        self.board = create_board()
        self.game_over = False
        self.current_player = HUMAN

        # Colors for display
        self.colors = {
            0: '⚪',  # Empty
            1: '🔴',  # Human (Red)
            2: '🟡'   # AI (Yellow)
        }

        self.setup_widgets()

    def setup_widgets(self):
        """Setup interactive widgets for the game."""
        # Create column buttons
        self.buttons = []
        button_layout = widgets.HBox([])

        for col in range(self.cols):
            btn = widgets.Button(
                description=f'Col {col}',
                button_style='info',
                layout=widgets.Layout(width='80px', height='40px')
            )
            btn.on_click(lambda b, c=col: self.make_move(c))
            self.buttons.append(btn)
            button_layout.children += (btn,)

        self.button_layout = button_layout

        # Status display
        self.status = widgets.HTML(
            value="<h3>🎮 Connect Four Game</h3><p>👤 Your turn! Click a column to drop your piece</p>",
            layout=widgets.Layout(width='100%')
        )

        # Board display
        self.board_display = widgets.HTML()

        # Control buttons
        self.new_game_btn = widgets.Button(
            description='🔄 New Game',
            button_style='success',
            layout=widgets.Layout(width='120px', height='40px')
        )
        self.new_game_btn.on_click(lambda b: self.new_game())

        self.quit_btn = widgets.Button(
            description='❌ Quit',
            button_style='danger',
            layout=widgets.Layout(width='120px', height='40px')
        )
        self.quit_btn.on_click(lambda b: self.quit_game())

        self.control_layout = widgets.HBox([self.new_game_btn, self.quit_btn])

    def display_board(self):
        """Display the current board state using HTML table."""
        board_html = "<table style='border-collapse: collapse; margin: 20px auto;'>"

        for row in range(self.rows):
            board_html += "<tr>"
            for col in range(self.cols):
                piece = self.colors[self.board[row][col]]
                board_html += f"<td style='border: 2px solid #333; width: 50px; height: 50px; text-align: center; font-size: 24px;'>{piece}</td>"
            board_html += "</tr>"

        board_html += "</table>"

        # Add column numbers
        board_html += "<div style='text-align: center; margin-top: 10px;'>"
        for col in range(self.cols):
            board_html += f"<span style='display: inline-block; width: 50px; text-align: center; font-weight: bold;'>{col}</span>"
        board_html += "</div>"

        self.board_display.value = board_html

    def make_move(self, col):
        """Make a move in the specified column."""
        if self.game_over or self.current_player != HUMAN:
            return

        # Check if valid move using existing function
        if not is_valid_location(self.board, col):
            self.status.value = "<h3>⚠️ Invalid Move!</h3><p>Column is full or out of range. Please try another column.</p>"
            return

        # Get row and drop piece using existing functions
        row = get_next_open_row(self.board, col)
        if row is None:
            return

        drop_piece(self.board, row, col, HUMAN)
        self.display_board()

        # Check for win using existing function
        if winning_move(self.board, HUMAN):
            self.game_over = True
            self.status.value = "<h3>🎉 CONGRATULATIONS! YOU WON! 🎉</h3><p>You got 4 in a row! Excellent strategy!</p>"
            return

        # Check for draw using existing function
        if board_full(self.board):
            self.game_over = True
            self.status.value = "<h3>🤝 IT'S A DRAW! 🤝</h3><p>The board is full - great game!</p>"
            return

        # Switch to computer
        self.current_player = AI
        self.status.value = "<h3>🤖 Computer is thinking...</h3><p>AI is calculating the best move...</p>"

        # Computer move after a short delay
        time.sleep(1)
        self.computer_move()

    def computer_move(self):
        """Computer makes a move using existing AI logic."""
        if self.game_over:
            return

        # Use existing minimax function
        _, col = minimax(self.board, depth=3, maximizing_player=True)
        if col is None:
            valid = get_valid_locations(self.board)
            col = random.choice(valid) if valid else 0

        row = get_next_open_row(self.board, col)
        if row is not None:
            drop_piece(self.board, row, col, AI)

        self.display_board()

        # Check for win using existing function
        if winning_move(self.board, AI):
            self.game_over = True
            self.status.value = "<h3>🤖 COMPUTER WINS! 🤖</h3><p>Better luck next time! The AI is quite smart!</p>"
            return

        # Check for draw using existing function
        if board_full(self.board):
            self.game_over = True
            self.status.value = "<h3>🤝 IT'S A DRAW! 🤝</h3><p>The board is full - great game!</p>"
            return

        # Switch back to human
        self.current_player = HUMAN
        self.status.value = "<h3>👤 Your turn!</h3><p>Click a column to drop your piece</p>"

    def is_board_full(self):
        """Check if board is full using existing function."""
        return board_full(self.board)

    def new_game(self):
        """Start a new game."""
        self.board = create_board()
        self.current_player = HUMAN
        self.game_over = False
        self.status.value = "<h3>🎮 New Game Started!</h3><p>👤 Your turn! Click a column to drop your piece</p>"
        self.display_board()

    def quit_game(self):
        """Quit the game."""
        self.status.value = "<h3>👋 Thanks for playing!</h3><p>Come back soon!</p>"
        self.game_over = True

    def show_game(self):
        """Display the complete game interface."""
        display(self.status)
        display(self.board_display)
        display(self.button_layout)
        display(self.control_layout)
        self.display_board()


## Start the GUI Game

Run the cell below to launch the beautiful GUI version!


In [4]:
# 🎮 MAIN GAME LAUNCHER - GUI VERSION
print("🎮 CONNECT FOUR - GUI VERSION")
print("=" * 50)
print("Features:")
print("• 🎨 Interactive widgets interface")
print("• 🖱️ Click buttons to drop pieces")
print("• 🤖 AI opponent with Minimax algorithm")
print("• 🏆 Win detection and highlighting")
print("• 🔄 New game and quit buttons")
print("• ☁️ Works perfectly in cloud environments!")
print("=" * 50)

# Create and show the game
gui_game = ConnectFourGUI()
gui_game.show_game()


🎮 CONNECT FOUR - GUI VERSION
Features:
• 🎨 Interactive widgets interface
• 🖱️ Click buttons to drop pieces
• 🤖 AI opponent with Minimax algorithm
• 🏆 Win detection and highlighting
• 🔄 New game and quit buttons
• ☁️ Works perfectly in cloud environments!


HTML(value='<h3>🎮 Connect Four Game</h3><p>👤 Your turn! Click a column to drop your piece</p>', layout=Layout(…

HTML(value='')